# Praktikum Pengantar Pembelajaran Mesin


---
## Bab 4. Klasifikasi Naive Bayes


Tuliskan Nama, NIM, dan kelas Anda:

Nama  : Amry Yahya

NIM   : 205150200111064

Kelas : PPM TIF-A

### 1) Import Data

Unduh dataset yang akan digunakan pada praktikum kali ini. Anda dapat menggunakan aplikasi wget untuk mendowload dataset dan menyimpannya dalam Google Colab. Jalankan cell di bawah ini untuk mengunduh dataset

In [283]:
# !wget https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

Setelah dataset berhasil diunduh, langkah berikutnya adalah membaca dataset dengan memanfaatkan fungsi **readcsv** dari library pandas. Lakukan pembacaan berkas csv ke dalam dataframe dengan nama **data** menggunakan fungsi **readcsv**. Jangan lupa untuk melakukan import library pandas terlebih dahulu


In [284]:
import pandas as pd
import numpy as np
data = pd.read_csv('horse_manual.csv')

In [285]:
data=data.drop(['age','temp_of_extremities','peripheral_pulse'],axis=1)



Cek isi dataset Anda dengan menggunakan perintah **head()**

In [286]:
data.head()

,rectal_temp,pulse,respiratory_rate,outcome
0,0.574074,0.233766,0.227273,died
1,0.703704,0.376623,0.136364,euthanized
2,0.537037,0.064935,0.181818,lived
3,0.685185,0.870130,0.863636,died
4,0.351852,0.480519,0.306818,died


## 2) Membagi data menjadi data latih dan data uji

Metode pembelajaran mesin memerlukan dua jenis data :


1.   Data latih : Digunakan untuk proses training metode klasifikasi
2.   Data uji : Digunakan untuk proses evaluasi metode klasifikasi

Data uji dan data latih perlu dibuat terpisah (mutualy exclusive) agar hasil evaluasi lebih akurat.

Data uji dan data latih dapat dibuat dengan cara membagi dataset dengan rasio tertentu, misalnya 80% data latih dan 20% data uji.

Library Scikit-learn memiliki fungsi [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) pada modul **model_selection** untuk membagi dataset menjadi data latih dan data uji. Bagilah dataset anda menjadi dua, yaitu **data_latih** dan **data_uji**.


In [287]:
data_latih = data[0:16]
data_uji = data [16:20]

Tampilkan banyaknya data pada **data_latih** dan **data_uji**. Seharusnya **data_latih** terdiri dari 120 data, dan **data_uji** terdiri dari 30 data

In [288]:
print(data_latih.shape[0])
print(data_uji.shape[0])

16
4


Pisahkan label/kategori dari data uji menjadi variabel tersendiri. Beri nama variabelnya  **label_uji**

In [289]:
label_uji = data_uji.pop('outcome')

## 3) Menghitung Prior

Tahapan pertama pada algoritma Naive Bayes adalah perhitungan prior. Prior suatu kelas merupakan peluang awal munculnya kelas tersebut. Atau dengan kata lain, prior merupakan frekuensi relatif dari suatu kelas terhadap keseluruhan data. 

Buatlah fungsi bernama **hitung_prior** yang berfungsi menghitung nilai prior dari masing-masing kelas yang terdapat pada data latih. Fungsi ini menerima input berupa list kelas dari data latih.

In [290]:
from collections import Counter
def hitung_prior(list_kelas):
  n_data = len(list_kelas)
  prior = Counter(list_kelas)
  for key in prior.keys():
    prior[key] = prior[key]/n_data
  return prior

Ujilah fungsi **hitung_prior** menggunakan kelas dari variabel **data_latih**. Amati hasilnya

In [291]:
prior = hitung_prior(data_latih['outcome'])
print(prior)

Counter({'lived': 0.5, 'died': 0.375, 'euthanized': 0.125})


## 4) Menghitung *likelihood*

Praktikum kali ini mengimplementasikan metode Naive Bayes menggunakan datset iris. Semua fitur pada dataset iris memiliki tipe numerik. Dengan demikian, perhitungan *likelihood* dilakukan menggunakan Gaussian Model.

Tahapan yang perlu dilakukan sebelum perhitungan *likelihood* adalah menghitung rata-rata dan deviasi standar masing-masing fitur per kelas. Buatlah fungsi **hitung_rata2_std_kelas** yang berfungsi menghitung rata-rata dan standar deviasi per fitur dan kelas

In [292]:
def hitung_rata2_std_kelas(input_data):
  list_columns = input_data.columns[:-1]
  class_columns_name = input_data.columns[-1]
  list_class = set(input_data[class_columns_name])
  rata2 = {}
  std = {}
  for column in list_columns:
    for a_class in list_class:
      rata2[(a_class, column)] = input_data.loc[input_data[class_columns_name] == a_class][column].mean()
      std[(a_class, column)] = input_data.loc[input_data[class_columns_name] == a_class][column].std()
  return (rata2, std)

Buatlah fungsi **hitung_likelihood_gaussian** yang bertujuan menghitung nilai likelihood suatu nilai terhadap rata-rata dan deviasi standar tertentu.

In [293]:
import math
def hitung_likelihood_gaussian(data, rata2, std):
  ans = (1/math.sqrt(2*math.pi*(std**2))) * math.exp((-1*((data-rata2)**2))/(2*(std**2)))
  return ans

## 5) Proses *training*
Proses training pada Naive Bayes dengan model Gaussian dilakukan untuk menghitung prior serta  rata-rata dan deviasi standar pada masing-msaing fitur dan kelas. 

In [294]:
def training_naive_bayes_gaussian(data_latih):
  class_column_name = data_latih.columns[-1]
  prior = hitung_prior(data_latih[class_column_name])
  rata2, std = hitung_rata2_std_kelas(data_latih)
  list_class = set(data_latih[class_column_name])
  list_columns = data_latih.columns[:-1]
  model = {}
  model['prior'] = prior
  model['rata2'] = rata2
  model['std'] = std
  model['list_class'] = list_class
  model['list_columns'] = list_columns
  return model

Lakukanlah proses training dan simpan hasil training pada variabel **model**

In [295]:
model = training_naive_bayes_gaussian(data_latih)

## 6) Proses *testing*
Proses testing dilakukan dengan menghitung nilai posterior dari data uji berdasarkan nilai model yang diperoleh saat training. Nilai posterior diperoleh dari perkalian prior dengan *likelihood* masing-masing fiturnya. Setelah itu, penentuan kelas data uji dilakukan berdasarkan nilai posterior terbesar

In [296]:
def testing_naive_bayes_gaussian(model, data_uji):
  prior = model['prior']
  rata2 = model['rata2']
  std = model['std']
  list_class = model['list_class']
  list_columns = model['list_columns']
  posterior = dict.fromkeys(list_class, 1)
  for a_class in list_class:
    print(a_class)
    for column in list_columns:
      likelihood = hitung_likelihood_gaussian(data_uji[column], rata2[(a_class, column)], std[(a_class, column)])
      posterior[a_class] = posterior[a_class] * likelihood
      print(column,'=',likelihood)
  kelas_uji = max(posterior, key=posterior.get)
  return kelas_uji

Ujilah fungsi **testing_naive_bayes_gaussian** menggunakan data uji pertama. Bandingkan hasilnya dengan nilai label sebenarnya.

In [308]:
testing_naive_bayes_gaussian(model, data_uji.iloc[0])

lived
rectal_temp = 4.113008659584946
pulse = 0.0032956451002718206
respiratory_rate = 1.9241380348346662
euthanized
rectal_temp = 2.2814693090532145
pulse = 7.914941693565258e-13
respiratory_rate = 2.4166520920724723
died
rectal_temp = 3.334353351448438
pulse = 1.1677025782314217
respiratory_rate = 1.5538571652824373


'died'

In [298]:
label_uji.iloc[0]

'died'

Lakukan pengujian untuk semua data uji. Selanjutnya bandingkan dengan label sebenarnya untuk menghitung berapa banyak data uji yang berhasil diprediksi dengan benar

In [299]:
yang_benar = 0
for i in range(data_uji.shape[0]):
  if (label_uji.iloc[i] == testing_naive_bayes_gaussian(model, data_uji.iloc[i])):
    yang_benar = yang_benar + 1

print(yang_benar/data_uji.shape[0])

lived
rectal_temp = 4.113008659584946
pulse = 0.0032956451002718206
respiratory_rate = 1.9241380348346662
euthanized
rectal_temp = 2.2814693090532145
pulse = 7.914941693565258e-13
respiratory_rate = 2.4166520920724723
died
rectal_temp = 3.334353351448438
pulse = 1.1677025782314217
respiratory_rate = 1.5538571652824373
lived
rectal_temp = 1.852715241046663
pulse = 2.5215959554177427
respiratory_rate = 2.674596507982576
euthanized
rectal_temp = 1.0124854692661212
pulse = 1.743381924812041e-08
respiratory_rate = 2.915038855784313
died
rectal_temp = 2.1262820737769728
pulse = 0.6927022668779322
respiratory_rate = 1.119427252132961
lived
rectal_temp = 2.315803465699878
pulse = 3.4646243261088205
respiratory_rate = 2.5451689509832103
euthanized
rectal_temp = 1.1924826088196447
pulse = 0.020965985717144472
respiratory_rate = 2.562483484354294
died
rectal_temp = 2.436906527451559
pulse = 1.114739823687847
respiratory_rate = 0.9824725633432053
lived
rectal_temp = 0.2503725656279097
pulse = 0.08

## TUGAS
Pada tugas kali ini Anda akan menggunakan dataset **[Car Evaluation Dataset](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation)** yang telah dimodifikasi. Fitur-fitur yang ada bertipe kategori sebagai berikut:
1. **buying**. Merupakan harga beli mobil. Nilai : vhigh, high, med, low
2. **maint**. Menandakan biaya perawatan mobil. Nilai: vhigh, high, med, low
3. **lug_boot**. Menandakan ukuran bagasi. Nilai: small, med, big
4. **safety**. Menandakan skor keamanan mobil. Nilai:  low, med, high
5. **class**. Merupakan kelas data. Nilai: unacc, acc, good, vgood

Download dataset dari https://dataset-ppm.s3.amazonaws.com/car_sample.csv dan letakkan dalam variabel **data_tugas**.

In [300]:
# !wget https://gist.githubusercontent.com/Arceister/a890983a3c3f151eebf5702d870d93a7/raw/a4ed5bf0d779a8c8b8cda7145c44e1a36fe8f7ca/car_evaluation.csv

In [301]:
import pandas as pd
data = pd.read_csv('car_evaluation.csv')
data.head()

FileNotFoundError: [Errno 2] No such file or directory: 'car_evaluation.csv'

Bagi data menjadi data latih dan uji dengan nama **tugas_latih** dan **tugas_uji** dengan rasio 70%:30%

In [ ]:
from sklearn.model_selection import train_test_split
tugas_latih, tugas_uji = train_test_split(data, test_size = 0.3)

print(tugas_latih.shape[0])
print(tugas_uji.shape[0])

1209
519


Pisahkan label/kategori dari data uji menjadi variabel tersendiri. Beri nama variabelnya **tugas_label_uji**

In [ ]:
tugas_label_uji = tugas_uji.pop('class')

Hitunglah nilai prior dari data latih dan simpan nilainya pada variabel bernama **prior_tugas**. Anda dapat menggunakan fungsi **hitung_prior** yang telah Anda buat.

In [ ]:
from collections import Counter
def hitung_prior(list_kelas):
  n_data = len(list_kelas)
  prior = Counter(list_kelas)
  for key in prior.keys():
    prior[key] = prior[key]/n_data
  return prior

prior_tugas = hitung_prior(tugas_latih['class']);
print(prior_tugas)

Counter({'unacc': 0.7129859387923904, 'acc': 0.2142266335814723, 'good': 0.04218362282878412, 'vgood': 0.030603804797353185})


Buatlah fungsi bernama **hitung_likelihood_multinomial** dengan parameter input berupa data latih. Fungsi ini bertujuan menghitung nilai likelihood pada masing-masing kolom pada data latih. Operasi-operasi yang perlu dilakukan pada fungsi ini adalah:


1.   Mendapatkan daftar kolom data
2.   Mendapatkan nama kolom kelas pada data (kolom terakhir pada data)
3.   Mendapatkan nama-nama kelas pada data (unik).
4.   Menghitung likelihood pada masing-masing kolom, nilai kolom, dan nilai kelas

Sebagai saran, Anda dapat menggunakan struktur data dictionary untuk menyimpan nilai likelihood. Key pada dictionary tersebut adalah tuple(nama kolom, nilai pada kolom, nama kelas). Petunjuk penggunaan dictionary dengan key berupa tuple dapat dibaca [di sini](https://stackoverflow.com/questions/1938614/in-what-case-would-i-use-a-tuple-as-a-dictionary-key).



In [ ]:
import numpy as np
def hitung_likelihood_multinomial(data_latih):
  #tulis kode Anda disini
  list_columns = data_latih.columns[:-1]
  class_columns_name = data_latih.columns[-1]
  list_class = set(data_latih[class_columns_name])
  likelihood={}
  for column in list_columns:
    for e in list_class:
      for val in np.unique(data_latih[column]):
        length_c = data_latih.loc[np.logical_and((data_latih[class_columns_name]==e), (data_latih[column]==val))]
        pembilang = length_c[column].count()
        penyebut = data_latih.loc[data_latih[class_columns_name]==e][column].count()
        likelihood[(column, val, e)] = pembilang/penyebut
  return likelihood

Ujilah fungsi **hitung_likelihood_multinomial** dengan parameter input berupa variabel **tugas_latih**

In [ ]:
likelihood = hitung_likelihood_multinomial(tugas_latih)
print(likelihood)

{('buying', 'high', 'unacc'): 0.24477958236658934, ('buying', 'low', 'unacc'): 0.21577726218097448, ('buying', 'med', 'unacc'): 0.2296983758700696, ('buying', 'vhigh', 'unacc'): 0.30974477958236657, ('buying', 'high', 'vgood'): 0.0, ('buying', 'low', 'vgood'): 0.6216216216216216, ('buying', 'med', 'vgood'): 0.3783783783783784, ('buying', 'vhigh', 'vgood'): 0.0, ('buying', 'high', 'acc'): 0.2625482625482625, ('buying', 'low', 'acc'): 0.2277992277992278, ('buying', 'med', 'acc'): 0.3359073359073359, ('buying', 'vhigh', 'acc'): 0.17374517374517376, ('buying', 'high', 'good'): 0.0, ('buying', 'low', 'good'): 0.6470588235294118, ('buying', 'med', 'good'): 0.35294117647058826, ('buying', 'vhigh', 'good'): 0.0, ('maint', 'high', 'unacc'): 0.26218097447795824, ('maint', 'low', 'unacc'): 0.2308584686774942, ('maint', 'med', 'unacc'): 0.22621809744779584, ('maint', 'vhigh', 'unacc'): 0.28074245939675174, ('maint', 'high', 'vgood'): 0.21621621621621623, ('maint', 'low', 'vgood'): 0.40540540540540

Buatlah fungsi bernama **training_naive_bayes_multinomial** dengan input berupa data latih. Fungsi ini bertujuan membentuk model yang berisi:


1.   Nilai prior semua kelas
2.   Nilai likelihood semua kolom, nilai kolom, dan kelas
3.   Daftar kelas pada data latih (unik)
4.   Daftar nama kolom pada data latih

Return value dari fungsi ini adalah **model**



In [ ]:
def training_naive_bayes_multinomial(data_latih):
  #tulis kode Anda disini
  class_columns_name = data_latih.columns[-1]
  prior_tugas = hitung_prior(data_latih[class_columns_name])
  likelihood = hitung_likelihood_multinomial(data_latih)
  list_class = set(data_latih[class_columns_name])
  list_columns = data_latih.columns[:-1]
  model = {}
  model['prior'] = prior_tugas
  model['likelihood'] = likelihood
  model['list_class'] = list_class
  model['list_columns'] = list_columns
  return model

Jalankan fungsi **training_naive_bayes_multinomial** dan simpan hasilnya dalam variabel **model_tugas**

In [ ]:
model_tugas = training_naive_bayes_multinomial(tugas_latih)

Buatlah fungsi **testing_naive_bayes_multinomial** untuk mendapatkan kelas dari sebuah data uji. Langkah yang dilakukan hampir sama dengan Gaussian, yaitu mengalikan likelihood semua kolom dengan posterior.

In [ ]:
def testing_naive_bayes_multinomial(model,data_uji):
  #tulis kode Anda disini
  prior = model['prior']
  likelihood = model['likelihood']
  list_class = model['list_class']
  list_columns = model['list_columns']
  posterior = dict.fromkeys(list_class, 1)
  for e in list_class:
    for column in list_columns:
      posterior[e] = posterior[e] * likelihood[column,data_uji[column],e]
  kelas_uji = max(posterior, key=posterior.get)
  return kelas_uji

Lakukan pengujian menggunakan sebuah data uji

In [ ]:
idx = 10
uji = tugas_uji.iloc[idx]
kelas_uji = testing_naive_bayes_multinomial(model_tugas,uji)
print(kelas_uji)
print(tugas_label_uji.iloc[idx])

unacc
unacc


Lakukan pengujian terhadap semua data uji.

In [ ]:
prediksi_total =[]
for indeks in range(tugas_uji.shape[0]):
  prediksi_total.append(testing_naive_bayes_multinomial(model_tugas,tugas_uji.iloc[indeks]))
print("Total prediksi benar: ",sum(prediksi_total==tugas_label_uji))

Total prediksi benar:  431


In [ ]:
len(tugas_label_uji)

519

In [ ]:
print('Presentase Benar',sum(prediksi_total==tugas_label_uji)/len(tugas_label_uji))

Presentase Benar 0.8304431599229287
